In [0]:

-- To cleanup tables
USE GENERAL_CATALOG.SILVER_SCHEMA;
DROP TABLE IF EXISTS silver_employee

In [0]:

CREATE TABLE IF NOT EXISTS SILVER_EMPLOYEE(employee_id int not null,first_name string,last_name string,email string,department string,hire_date string)

In [0]:
insert into silver_employee values(101, "Alice", "Smith", "alice.s@example.com", "Sales", "2020-01-15"),
    (102, "Bob", "Johnson", "bob.j@example.com", "Marketing", "2019-03-22"),
    (103, "Charlie", "Brown", "charlie.b@example.com", "Engineering", "2021-06-01"),
    (104, "Diana", "Prince", "diana.p@example.com", "HR", "2018-11-10")

In [0]:
select * from silver_employee

In [0]:
create temp view silver_employee_view as select *,current_timestamp as created_at,current_timestamp as updated_at from silver_employee

In [0]:
select * from silver_employee_view

In [0]:
-- Use the target catalog and schema
USE CATALOG general_catalog;
USE SCHEMA silver_schema;

-- Create or overwrite the managed Delta table from the temporary view
CREATE OR REPLACE TABLE silver_employee
USING DELTA
AS
SELECT * FROM silver_employee_view

In [0]:
select * from silver_employee

In [0]:
USE CATALOG general_catalog;
USE SCHEMA silver_schema;
create temp view silver_employee_incrimental_view as select *,current_timestamp as created_at,current_timestamp as updated_at  
FROM VALUES
  (101, 'Alice', 'Smith', 'alice.smith.new@example.com', 'Marketing', '2020-01-15'),
  (102, 'Bob', 'Johnson', 'bobjohnson@example.com', 'Marketing', '2019-03-22'),
  (105, 'Emily', 'Clark', 'emily.c@example.com', 'Finance', '2022-09-01')
AS t(employee_id, first_name, last_name, email, department, hire_date);

In [0]:
select * from silver_employee_incrimental_view

In [0]:
USE CATALOG general_catalog;
USE SCHEMA silver_schema;

-- Perform MERGE (UPSERT)
MERGE INTO silver_employee AS target
USING silver_employee_incrimental_view AS source
ON target.employee_id = source.employee_id

-- Update existing records
WHEN MATCHED THEN UPDATE SET
  target.first_name   = source.first_name,
  target.last_name    = source.last_name,
  target.email        = source.email,
  target.department   = source.department,
  target.hire_date    = source.hire_date,
  target.updated_at   = source.updated_at

-- Insert new records
WHEN NOT MATCHED THEN INSERT (
  employee_id, first_name, last_name, email, department, hire_date, created_at, updated_at
)
VALUES (
  source.employee_id, source.first_name, source.last_name, source.email,
  source.department, source.hire_date, source.updated_at, source.updated_at
);

---- First MERGE operation completed.

In [0]:

select * from general_catalog.silver_schema.silver_employee order by employee_id 